# Exploratory Data Analysis

In this notebook, we will be importing the data and conducting EDA to evaluate our most pertinent features.

***

# Initialization

Importing libraries and setting seeds.

In [ ]:
# Setting PYTHONHASHSEED
import os

pyhashseed1 = os.environ.get('PYTHONHASHSEED')
os.environ['PYTHONHASHSEED'] = '0'
pyhashseed2 = os.environ.get('PYTHONHASHSEED')

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    print('Make sure the following says \'None\': ', pyhashseed1)
    print('Make sure the following says \'0\': ', pyhashseed2)

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

# Setting seed
np.random.seed(42)

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / 'plots'

def save_fig(fig_name, tight_layout=True, fig_extension='png', resolution=300):
    '''Saves an image to the plots folder with the specified name.'''
    path = IMAGES_PATH / f'{fig_name}.{fig_extension}'
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
def plt_show(plt_name):
    '''Saves an image using save_fig() under the plt_name and displays it.'''
    save_fig(plt_name)
    plt.show()

In [ ]:
# Importing data
adclicks = pd.read_csv('data/ad_click_dataset.csv')

***

# Overview of data structure

We will be examining the data and its underlying structure

In [ ]:
# Examining the top 10 rows
adclicks.head(10)

In [ ]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Checking duplicates
    duplicates = adclicks.duplicated()
    print(len(adclicks[duplicates]))
    
# Dropping duplicates
adclicks = adclicks.drop_duplicates()

In [ ]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # Information on features
    adclicks.info()

Above, we can see that there are quite a bit of missing information (NaNs) and we also had to remove a significant amount of duplicate data.

A summary of `adclicks.info()`:

- There are 7,147 instances.
- `age`, `gender`, `device_type`, `ad_position`, `browsing_history`, and `time_of_day` have missing values.
- Most of the features are non-numeric, possibly categorical.
- `full_name` seems to be an irrelevant feature, as all the names are anonymized. I will remove this feature.
    - I will also check if `id` is a unique identifier, if so, I will drop it as use rowID as an identifier instead.

In [ ]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # The number of unique IDs
    print(adclicks['id'].nunique())

Since `id` is not unique for all rows, we will keep it and keep in mind this fact when we split our test set.

In [ ]:
# Dropping `full_name`
adclicks = adclicks.drop(columns='full_name')

***

## Examining categorical features

Let's examine what the possible values our non-numerical features have.

In [ ]:
def categorical_features(df):
    '''Returns a list of the names of the cateogorical columns in a pandas dataframe.'''
    return list(df.select_dtypes(include=['object', 'category', 'bool']).columns)

In [ ]:
# Possible values of the categorical features
categoricals = categorical_features(adclicks)

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # Check the columns in categoricals
    display(categoricals)

    # Display the unique values for each categorical feature
    for feature in categoricals:
        print(f'{feature}: ' ,list(adclicks[feature].unique()))

Each categorical feature represents the following:

- `gender`: the gender of the user (Male, Female, Non-Binary).
- `device_type`: the type of device of the user when viewing the ad (Mobile, Desktop, Tablet).
- `ad_position`: the position of the ad on the webpage (Top, Side, Bottom).
- `browsing_history`: the user's browsing activity prior to seeing the ad (Shopping, News, Entertainment, Education, Social Media).
- `time_of_day`: the time when the user viewed the ad (Morning, Afternoon, Evening, Night).

Our numerical features represent the following:

- `id`: unique ID for each user (NOT EACH ROW).
    - This is important to note as we want to be aware of this when we split our test set in order to avoid data leakage.
- `age`: the age of each user.
- `click`: the **target label** indicating whether the user clicked on the ad (Binary).

***

# Splitting an initial training set for EDA + transformations

We will be splitting an initial training set to evaluate for EDA and assess what the necessary transformations will be.

In [ ]:
# Splitting a random train-test set
from sklearn.model_selection import train_test_split
X = adclicks.drop(columns='click')
y = adclicks['click']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

***

# Pre-split EDA

We will perform some light EDA before we split to understand the overall pattern of the data and how it compares to our training data.

In [ ]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # Display value counts for categorical features
    print('VALUE COUNTS OF CATEGORICAL FEATURES')
    for category in categoricals:
        display(X_train[category].value_counts())
        
    # Description statistics
    print('\nDESCRIPTION STATISTICS')
    display(X_train.describe())

In [ ]:
import missingno as msno

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # Evaluate missing data
    missing_values = X_train[X_train.isnull().any(axis=1)]
    print(len(missing_values))
    
    # Show the missing matrix
    msno.matrix(X_train)
    plt_show('missing_matrix')
    
    # Show the nullity correlation heatmap
    msno.heatmap(X_train)
    plt_show('nullity_corr_heatmap')

From a small look at our data, the following can be observed:

- The classes of each categorical feature are balanced
- The average age is 40 years old, and the data seems to not be skewed as the median also aligns with this value
- Most of the instances have missing data, which will significantly affect our modeling.
    - Luckily, the data shows to be missing at random (MAR), based on the sporadic pattern of the missing matrix and the neutral nullity correlation heatmap, so we can be optimistic about not running into issues during imputation of missing values.

***

# Splitting the test set

Considering that our target variable `click` is categorical, I am thinking that a **stratified-split** would be the best way to split our training and test data in order to represent our classes well. We will also have to split based on `id` to ensure that the same user is not in both the training and test set.

First, I want to see if we have any class imbalances in the `id` column.

In [ ]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # Check the balance of user id
    max_appearance = adclicks['id'].value_counts().max()
    display(max_appearance)

The most that any user appears is 14 times, which is not bad news for our splitting. We will proceed!

In [ ]:
# Splitting our data
from sklearn.model_selection import StratifiedGroupKFold

X = adclicks.drop(columns=['click']).copy()
y = adclicks['click'].copy()
groups = adclicks['id']

sgkf = StratifiedGroupKFold(n_splits=5, shuffle=False)
# Setting shuffle to false so that the indices stay consistent in the split
folds = []

for train_index, test_index in sgkf.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    folds.append([X_train, X_test, y_train, y_test])

In [ ]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # Checking that our split is sound
    for fold, i in zip(folds, range(0, len(folds))):
        X_train, X_test, y_train, y_test = fold
        
        train_users = set(X_train['id'])
        test_users = set(X_test['id'])
        
        # Printing output
        print(f'No overlap in train and test users in split {i + 1}:', train_users.isdisjoint(test_users))
        
        print('Train label values:')
        print(y_train.unique())
        print('Test label values:')
        print(y_test.unique(), '\n')

Great! We have no overlap in our test and train users and our train and test sets both have representation of our target variable `click`. Just as a precaution for when we implement our models later, I am going to calculate the "sample weight" of each user ID in the data (below).

In [ ]:
# Calculating sample weight in the training data
from sklearn.utils.class_weight import compute_sample_weight

for fold in folds:
    train_set = fold[0].copy()
    sample_weight = compute_sample_weight(class_weight='balanced', y=train_set['id'])
    train_set.loc[:, 'sample_weight'] = sample_weight
    fold[0] = train_set

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    # Display for the first split
    display(folds[0][0].head())

***

# Next steps

Now, with successfully split data, we will move on to transforming and preprocessing. While EDA would usually occur during this time, since we split our training data into multiple splits, there's a chance that evaluating each split would result in data leakage. In order to avoid this, we will just proceed and apply any imputations that were fit to the original random training set that we split for EDA.